### 190 original month_lag で期間を区切り集計を行う
相対的に時系列のズレはあるが、初期3ヶ月のトランザクションを共通集計し、targetの期間と一致すると考えられるグループで予測モデルを作り、他のグループはその予測モデルで初回のロイヤリティを予測する

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.38it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [3]:
df = df_hist

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth1.shape)
print(auth0.shape)

cat1_0 = False
# cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]
#     del auth1, auth0
#     gc.collect()

(26595452, 21)
(2516909, 21)


In [22]:
term_type = 'all'
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)

if cat1_0:
    df_list = [auth1_cat1, auth1_cat0, auth0_cat1, auth0_cat0, new_cat1, new_cat0]
    fname_list = [f'{term_type}_auth1_cat1', f'{term_type}_auth1_cat0', f'{term_type}_auth0_cat1', f'{term_type}_auth0_cat0', f'{term_type}_new_cat1', f'{term_type}_new_cat0']
else:
    df_list = [auth1, auth0, df_new]
    fname_list = [f'{term_type}_auth1', f'{term_type}_auth0', f'{term_type}_new']


def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


# def feat_agg(df, fname):
for df, fname in zip(df_list, fname_list):
    global train_test
    
#     if fname.count('auth1'):continue

    df = pd.get_dummies(df, columns=['category_2', 'category_3'])
    
#     aggs = {}
    aggs = {
    'category_1': ['sum', 'mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    }
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
#     aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
    aggs['all_term'] = ['max']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_freq_per_all_term'] = df_agg[f'{fname}_card_id_size'] / df_agg[f'{fname}_all_term_max']
    df_agg[f'{fname}_amount_per_all_term'] = df_agg[f'{fname}_purchase_amount_sum'] / df_agg[f'{fname}_all_term_max']
    df_agg[f'{fname}_installments_per_all_term'] = df_agg[f'{fname}_installments_sum'] / df_agg[f'{fname}_all_term_max']
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    df_agg.drop(f'{fname}_all_term_max', axis=1, inplace=True)
    
    # Monthly Agg
    del aggs['all_term']
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    month_agg.reset_index(drop=False,inplace=True)
    
    train_test = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
#     train_test = train_test.join(month_agg.set_index(key))
    del df_agg
    gc.collect()
    
    print(fname)
    print(train_test.shape)
    display(train_test.head())

100%|██████████| 3/3 [00:00<00:00, 140.31it/s]


all_auth1
(325540, 72)


,feature_1,feature_2,feature_3,first_active_month,target,all_auth1_category_1_sum,all_auth1_category_1_mean,all_auth1_category_2_1.0_mean,all_auth1_category_2_2.0_mean,all_auth1_category_2_3.0_mean,all_auth1_category_2_4.0_mean,all_auth1_category_2_5.0_mean,all_auth1_category_3_A_mean,all_auth1_category_3_B_mean,all_auth1_category_3_C_mean,all_auth1_month_lag_mean,all_auth1_month_lag_std,all_auth1_yyyy_week_nunique,all_auth1_purchase_date_max,all_auth1_purchase_date_min,all_auth1_month_diff_mean,all_auth1_month_diff_std,all_auth1_purchase_amount_sum,all_auth1_purchase_amount_max,all_auth1_purchase_amount_min,all_auth1_purchase_amount_mean,all_auth1_installments_sum,all_auth1_installments_max,all_auth1_installments_min,all_auth1_installments_mean,all_auth1_installments_std,all_auth1_merchant_id_nunique,all_auth1_merchant_category_id_nunique,all_auth1_card_id_size,all_auth1_city_id_nunique,all_auth1_state_id_nunique,all_auth1_subsector_id_nunique,all_auth1_freq_per_all_term,all_auth1_amount_per_all_term,all_auth1_installments_per_all_term,all_auth1_amount_per_installments_sum,all_auth1_amount_per_installments_mean,all_auth1_monthly_avg_category_1_sum,all_auth1_monthly_avg_category_1_mean,all_auth1_monthly_avg_category_2_1.0_mean,all_auth1_monthly_avg_category_2_2.0_mean,all_auth1_monthly_avg_category_2_3.0_mean,all_auth1_monthly_avg_category_2_4.0_mean,all_auth1_monthly_avg_category_2_5.0_mean,all_auth1_monthly_avg_category_3_A_mean,all_auth1_monthly_avg_category_3_B_mean,all_auth1_monthly_avg_category_3_C_mean,all_auth1_monthly_avg_month_lag_mean,all_auth1_monthly_avg_month_lag_std,all_auth1_monthly_avg_yyyy_week_nunique,all_auth1_monthly_avg_month_diff_mean,all_auth1_monthly_avg_month_diff_std,all_auth1_monthly_avg_purchase_amount_sum,all_auth1_monthly_avg_purchase_amount_max,all_auth1_monthly_avg_purchase_amount_min,all_auth1_monthly_avg_purchase_amount_mean,all_auth1_monthly_avg_installments_sum,all_auth1_monthly_avg_installments_max,all_auth1_monthly_avg_installments_min,all_auth1_monthly_avg_installments_mean,all_auth1_monthly_avg_installments_std,all_auth1_monthly_avg_merchant_id_nunique,all_auth1_monthly_avg_merchant_category_id_nunique,all_auth1_monthly_avg_card_id_size,all_auth1_monthly_avg_city_id_nunique,all_auth1_monthly_avg_state_id_nunique,all_auth1_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,0.0,0.000000,0.987854,0.0,0.0,0.000000,0.012146,1.0,0.000000,0.000000,-3.882591,2.429155,35,2018-02-25 09:31:15,2017-06-27 14:18:08,192.720648,75.202483,26.093750,2.000000,0.007523,0.105591,0.0,0.0,0.0,0.000000,0.000000,93,41,247,7,3,21,0.739521,0.078125,0.000000,26.093750,0.105591,0.000000,0.000000,0.984127,0.0,0.0,0.000000,0.015873,1.0,0.000000,0.000000,-4.000000,0.0,4.555556,195.020176,7.607743,2.898438,0.476318,0.014229,0.094421,0.000000,0.0,0.000000,0.000000,0.000000,17.666667,11.555556,27.444444,1.777778,1.222222,8.111111
C_ID_3d0044924f,4,1,0,2017-01,0.392913,29.0,0.085546,0.914454,0.0,0.0,0.000000,0.000000,0.0,0.802360,0.191740,-5.050147,3.836969,54,2018-01-31 22:31:09,2017-01-06 16:29:42,257.672566,117.798176,40.937500,2.000000,0.004517,0.120789,503.0,10.0,0.0,1.483398,1.341797,141,57,339,9,3,24,0.698969,0.084407,1.037113,0.081238,0.048615,2.230769,0.095850,0.904150,0.0,0.0,0.000000,0.000000,0.0,0.775157,0.221625,-6.000000,0.0,4.692308,286.396756,8.331325,3.150391,0.997559,0.009949,0.141235,38.687500,6.0,0.846191,1.622070,1.357422,18.461538,13.461538,26.076923,2.846154,1.692308,10.076923
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,0.0,0.000000,0.097561,0.0,0.0,0.000000,0.902439,1.0,0.000000,0.000000,-8.487805,3.893083,24,2018-02-27 19:08:25,2017-01-11 08:21:22,329.292683,118.713572,2.794922,0.601074,0.016785,0.068176,0.0,0.0,0.0,0.000000,0.000000,13,8,41,5,2,7,0.064263,0.004381,0.000000,2.794922,0.068176,0.000000,0.000000,0.058333,0.0,0.0,0.000000,0.941667,1.0,0.000000,0.000000,-6.666667,0.0,2.000000,275.127778,6.4

all_auth0
(325540, 139)


,feature_1,feature_2,feature_3,first_active_month,target,all_auth1_category_1_sum,all_auth1_category_1_mean,all_auth1_category_2_1.0_mean,all_auth1_category_2_2.0_mean,all_auth1_category_2_3.0_mean,all_auth1_category_2_4.0_mean,all_auth1_category_2_5.0_mean,all_auth1_category_3_A_mean,all_auth1_category_3_B_mean,all_auth1_category_3_C_mean,all_auth1_month_lag_mean,all_auth1_month_lag_std,all_auth1_yyyy_week_nunique,all_auth1_purchase_date_max,all_auth1_purchase_date_min,all_auth1_month_diff_mean,all_auth1_month_diff_std,all_auth1_purchase_amount_sum,all_auth1_purchase_amount_max,all_auth1_purchase_amount_min,all_auth1_purchase_amount_mean,all_auth1_installments_sum,all_auth1_installments_max,all_auth1_installments_min,all_auth1_installments_mean,all_auth1_installments_std,all_auth1_merchant_id_nunique,all_auth1_merchant_category_id_nunique,all_auth1_card_id_size,all_auth1_city_id_nunique,all_auth1_state_id_nunique,all_auth1_subsector_id_nunique,all_auth1_freq_per_all_term,all_auth1_amount_per_all_term,all_auth1_installments_per_all_term,all_auth1_amount_per_installments_sum,all_auth1_amount_per_installments_mean,all_auth1_monthly_avg_category_1_sum,all_auth1_monthly_avg_category_1_mean,all_auth1_monthly_avg_category_2_1.0_mean,all_auth1_monthly_avg_category_2_2.0_mean,all_auth1_monthly_avg_category_2_3.0_mean,all_auth1_monthly_avg_category_2_4.0_mean,all_auth1_monthly_avg_category_2_5.0_mean,all_auth1_monthly_avg_category_3_A_mean,all_auth1_monthly_avg_category_3_B_mean,all_auth1_monthly_avg_category_3_C_mean,all_auth1_monthly_avg_month_lag_mean,all_auth1_monthly_avg_month_lag_std,all_auth1_monthly_avg_yyyy_week_nunique,all_auth1_monthly_avg_month_diff_mean,all_auth1_monthly_avg_month_diff_std,all_auth1_monthly_avg_purchase_amount_sum,all_auth1_monthly_avg_purchase_amount_max,all_auth1_monthly_avg_purchase_amount_min,all_auth1_monthly_avg_purchase_amount_mean,all_auth1_monthly_avg_installments_sum,all_auth1_monthly_avg_installments_max,all_auth1_monthly_avg_installments_min,all_auth1_monthly_avg_installments_mean,all_auth1_monthly_avg_installments_std,all_auth1_monthly_avg_merchant_id_nunique,all_auth1_monthly_avg_merchant_category_id_nunique,all_auth1_monthly_avg_card_id_size,all_auth1_monthly_avg_city_id_nunique,all_auth1_monthly_avg_state_id_nunique,all_auth1_monthly_avg_subsector_id_nunique,all_auth0_category_1_sum,all_auth0_category_1_mean,all_auth0_category_2_1.0_mean,all_auth0_category_2_2.0_mean,all_auth0_category_2_3.0_mean,all_auth0_category_2_4.0_mean,all_auth0_category_2_5.0_mean,all_auth0_category_3_A_mean,all_auth0_category_3_B_mean,all_auth0_category_3_C_mean,all_auth0_month_lag_mean,all_auth0_month_lag_std,all_auth0_yyyy_week_nunique,all_auth0_purchase_date_max,all_auth0_purchase_date_min,all_auth0_month_diff_mean,all_auth0_month_diff_std,all_auth0_purchase_amount_sum,all_auth0_purchase_amount_max,all_auth0_purchase_amount_min,all_auth0_purchase_amount_mean,all_auth0_installments_sum,all_auth0_installments_max,all_auth0_installments_min,all_auth0_installments_mean,all_auth0_installments_std,all_auth0_merchant_id_nunique,all_auth0_merchant_category_id_nunique,all_auth0_card_id_size,all_auth0_city_id_nunique,all_auth0_state_id_nunique,all_auth0_subsector_id_nunique,all_auth0_freq_per_all_term,all_auth0_amount_per_all_term,all_auth0_installments_per_all_term,all_auth0_amount_per_installments_sum,all_auth0_amount_per_installments_mean,all_auth0_monthly_avg_category_1_sum,all_auth0_monthly_avg_category_1_mean,all_auth0_monthly_avg_category_2_1.0_mean,all_auth0_monthly_avg_category_2_2.0_mean,all_auth0_monthly_avg_category_2_3.0_mean,all_auth0_monthly_avg_category_2_4.0_mean,all_auth0_monthly_avg_category_2_5.0_mean,all_auth0_monthly_avg_category_3_A_mean,all_auth0_monthly_avg_category_3_B_mean,all_auth0_monthly_avg_category_3_C_mean,all_auth0_monthly_avg_month_lag_mean,all_auth0_monthly_avg_month_lag_std,all_auth0_monthly_avg_yyyy_week_nunique,all_auth0_monthly_avg_month_diff_mean,all_auth0_monthly_avg_month_diff_st

all_new
(325540, 206)


,feature_1,feature_2,feature_3,first_active_month,target,all_auth1_category_1_sum,all_auth1_category_1_mean,all_auth1_category_2_1.0_mean,all_auth1_category_2_2.0_mean,all_auth1_category_2_3.0_mean,all_auth1_category_2_4.0_mean,all_auth1_category_2_5.0_mean,all_auth1_category_3_A_mean,all_auth1_category_3_B_mean,all_auth1_category_3_C_mean,all_auth1_month_lag_mean,all_auth1_month_lag_std,all_auth1_yyyy_week_nunique,all_auth1_purchase_date_max,all_auth1_purchase_date_min,all_auth1_month_diff_mean,all_auth1_month_diff_std,all_auth1_purchase_amount_sum,all_auth1_purchase_amount_max,all_auth1_purchase_amount_min,all_auth1_purchase_amount_mean,all_auth1_installments_sum,all_auth1_installments_max,all_auth1_installments_min,all_auth1_installments_mean,all_auth1_installments_std,all_auth1_merchant_id_nunique,all_auth1_merchant_category_id_nunique,all_auth1_card_id_size,all_auth1_city_id_nunique,all_auth1_state_id_nunique,all_auth1_subsector_id_nunique,all_auth1_freq_per_all_term,all_auth1_amount_per_all_term,all_auth1_installments_per_all_term,all_auth1_amount_per_installments_sum,all_auth1_amount_per_installments_mean,all_auth1_monthly_avg_category_1_sum,all_auth1_monthly_avg_category_1_mean,all_auth1_monthly_avg_category_2_1.0_mean,all_auth1_monthly_avg_category_2_2.0_mean,all_auth1_monthly_avg_category_2_3.0_mean,all_auth1_monthly_avg_category_2_4.0_mean,all_auth1_monthly_avg_category_2_5.0_mean,all_auth1_monthly_avg_category_3_A_mean,all_auth1_monthly_avg_category_3_B_mean,all_auth1_monthly_avg_category_3_C_mean,all_auth1_monthly_avg_month_lag_mean,all_auth1_monthly_avg_month_lag_std,all_auth1_monthly_avg_yyyy_week_nunique,all_auth1_monthly_avg_month_diff_mean,all_auth1_monthly_avg_month_diff_std,all_auth1_monthly_avg_purchase_amount_sum,all_auth1_monthly_avg_purchase_amount_max,all_auth1_monthly_avg_purchase_amount_min,all_auth1_monthly_avg_purchase_amount_mean,all_auth1_monthly_avg_installments_sum,all_auth1_monthly_avg_installments_max,all_auth1_monthly_avg_installments_min,all_auth1_monthly_avg_installments_mean,all_auth1_monthly_avg_installments_std,all_auth1_monthly_avg_merchant_id_nunique,all_auth1_monthly_avg_merchant_category_id_nunique,all_auth1_monthly_avg_card_id_size,all_auth1_monthly_avg_city_id_nunique,all_auth1_monthly_avg_state_id_nunique,all_auth1_monthly_avg_subsector_id_nunique,all_auth0_category_1_sum,all_auth0_category_1_mean,all_auth0_category_2_1.0_mean,all_auth0_category_2_2.0_mean,all_auth0_category_2_3.0_mean,all_auth0_category_2_4.0_mean,all_auth0_category_2_5.0_mean,all_auth0_category_3_A_mean,all_auth0_category_3_B_mean,all_auth0_category_3_C_mean,all_auth0_month_lag_mean,all_auth0_month_lag_std,all_auth0_yyyy_week_nunique,all_auth0_purchase_date_max,all_auth0_purchase_date_min,all_auth0_month_diff_mean,all_auth0_month_diff_std,all_auth0_purchase_amount_sum,all_auth0_purchase_amount_max,all_auth0_purchase_amount_min,all_auth0_purchase_amount_mean,all_auth0_installments_sum,all_auth0_installments_max,all_auth0_installments_min,all_auth0_installments_mean,all_auth0_installments_std,all_auth0_merchant_id_nunique,all_auth0_merchant_category_id_nunique,...,all_auth0_installments_per_all_term,all_auth0_amount_per_installments_sum,all_auth0_amount_per_installments_mean,all_auth0_monthly_avg_category_1_sum,all_auth0_monthly_avg_category_1_mean,all_auth0_monthly_avg_category_2_1.0_mean,all_auth0_monthly_avg_category_2_2.0_mean,all_auth0_monthly_avg_category_2_3.0_mean,all_auth0_monthly_avg_category_2_4.0_mean,all_auth0_monthly_avg_category_2_5.0_mean,all_auth0_monthly_avg_category_3_A_mean,all_auth0_monthly_avg_category_3_B_mean,all_auth0_monthly_avg_category_3_C_mean,all_auth0_monthly_avg_month_lag_mean,all_auth0_monthly_avg_month_lag_std,all_auth0_monthly_avg_yyyy_week_nunique,all_auth0_monthly_avg_month_diff_mean,all_auth0_monthly_avg_month_diff_std,all_auth0_monthly_avg_purchase_amount_sum,all_auth0_monthly_avg_purchase_amount_max,all_auth0_monthly_avg_purchase_amount_min,all_auth0_monthly_avg_purchase_am

In [24]:
# impute

def impute_agg(df):
    for col in df.columns:
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df

df_feat = train_test
df_feat = impute_agg(df_feat)

In [26]:
if cat1_0:
    fname_list = [f'{term_type}_auth1_cat1', f'{term_type}_auth1_cat0', f'{term_type}_auth0_cat1', f'{term_type}_auth0_cat0', f'{term_type}_new_cat1', f'{term_type}_new_cat0']
else:
    fname_list = [f'{term_type}_auth1', f'{term_type}_auth0', f'{term_type}_new']

    
for fname in fname_list:

    df_feat['first_active_month'] =  pd.to_datetime(df_feat['first_active_month'])
    for col in df_feat.columns:
        if col.count('purchase_date'):
            df_feat[col] = pd.to_datetime( df_feat[col])
    
    # 最終のcat0, auth1までの期間
    df_feat[f'{fname}_term_from_first_month'] = (df_feat[f'{fname}_purchase_date_max'] - df_feat[f'first_active_month']).dt.days
    
    # date_max~date_minの期間
    df_feat[f'{fname}_term'] = (df_feat[f'{fname}_purchase_date_max'] - df_feat[f'{fname}_purchase_date_min']).dt.days
    
    # {term_type}の初回購買までの期間
#     df_feat[f'{term_type}_cat0_first_buy'] = (df_feat[f'{term_type}_auth1_cat0_purchase_date_min'] - df_feat[f'first_active_month']).dt.days
    
    # 20180501までの期間を各データセットパターンで
    df_feat[f'{fname}_term_from_now'] = (pd.to_datetime('2018-05-01') - df_feat[f'{fname}_purchase_date_max']).dt.days
    
    
# auth1, auth0の間の期間
if cat1_0:
    df_feat[f'{term_type}_auth1_0_cat0_term'] = df_feat[f'{term_type}_auth1_cat0_term_from_now'] - df_feat[f'{term_type}_auth0_cat0_term_from_now']
    df_feat[f'{term_type}_auth1_0_cat1_term'] = df_feat[f'{term_type}_auth1_cat1_term_from_now'] - df_feat[f'{term_type}_auth0_cat1_term_from_now']
else:
    df_feat[f'{term_type}_auth1_0_term'] = df_feat[f'{term_type}_auth1_term_from_now'] - df_feat[f'{term_type}_auth0_term_from_now']
    df_feat[f'{term_type}_auth1_0_term'] = df_feat[f'{term_type}_auth1_term_from_now'] - df_feat[f'{term_type}_auth0_term_from_now']
    df_feat[f'{term_type}_new_auth1_term'] = df_feat[f'{term_type}_new_term_from_now'] - df_feat[f'{term_type}_auth1_term_from_now']
    df_feat[f'{term_type}_auth1_0_purchase_date_max_diff'] = (df_feat[f'all_auth1_purchase_date_max'] - df_feat[f'all_auth0_purchase_date_max']).dt.days
    df_feat[f'{term_type}_auth1_0_purchase_date_min_diff'] = (df_feat[f'all_auth1_purchase_date_min'] - df_feat[f'all_auth0_purchase_date_min']).dt.days
    df_feat[f'{term_type}_new_min_auth1_max_date_diff'] = (df_feat[f'all_new_purchase_date_min'] - df_feat[f'all_auth1_purchase_date_max']).dt.days
    df_feat[f'{term_type}_new_min_auth1_min_date_diff'] = (df_feat[f'all_new_purchase_date_min'] - df_feat[f'all_auth1_purchase_date_min']).dt.days

In [30]:
# common feature
df_feat[f'first_active_month'].fillna('2017-03', inplace=True)
df_feat[f'first_active_month'] =  pd.to_datetime(df_feat['first_active_month'])

df_feat[f'{term_type}_elapsed_time'] = (pd.to_datetime('2018-05-01') - df_feat['first_active_month']).dt.days
date_min_cols = [col for col in df_feat.columns if col.count('purchase_date_min') and not(col.count('diff'))]
for col in date_min_cols:
    fname = ''
    if col.count('auth1'):
        fname += 'auth1_'
    elif col.count('auth0'):
        fname += 'auth0_'
    if col.count('new'):
        fname += 'new_'
    if col.count('cat1'):
        fname += 'cat1_'
    elif col.count('cat0'):
        fname += 'cat0_'
    df_feat[f'{term_type}_{fname}first_buy'] = (df_feat[col] - df_feat[f'first_active_month']).dt.days
df_feat.head()

,feature_1,feature_2,feature_3,first_active_month,target,all_auth1_category_1_sum,all_auth1_category_1_mean,all_auth1_category_2_1.0_mean,all_auth1_category_2_2.0_mean,all_auth1_category_2_3.0_mean,all_auth1_category_2_4.0_mean,all_auth1_category_2_5.0_mean,all_auth1_category_3_A_mean,all_auth1_category_3_B_mean,all_auth1_category_3_C_mean,all_auth1_month_lag_mean,all_auth1_month_lag_std,all_auth1_yyyy_week_nunique,all_auth1_purchase_date_max,all_auth1_purchase_date_min,all_auth1_month_diff_mean,all_auth1_month_diff_std,all_auth1_purchase_amount_sum,all_auth1_purchase_amount_max,all_auth1_purchase_amount_min,all_auth1_purchase_amount_mean,all_auth1_installments_sum,all_auth1_installments_max,all_auth1_installments_min,all_auth1_installments_mean,all_auth1_installments_std,all_auth1_merchant_id_nunique,all_auth1_merchant_category_id_nunique,all_auth1_card_id_size,all_auth1_city_id_nunique,all_auth1_state_id_nunique,all_auth1_subsector_id_nunique,all_auth1_freq_per_all_term,all_auth1_amount_per_all_term,all_auth1_installments_per_all_term,all_auth1_amount_per_installments_sum,all_auth1_amount_per_installments_mean,all_auth1_monthly_avg_category_1_sum,all_auth1_monthly_avg_category_1_mean,all_auth1_monthly_avg_category_2_1.0_mean,all_auth1_monthly_avg_category_2_2.0_mean,all_auth1_monthly_avg_category_2_3.0_mean,all_auth1_monthly_avg_category_2_4.0_mean,all_auth1_monthly_avg_category_2_5.0_mean,all_auth1_monthly_avg_category_3_A_mean,all_auth1_monthly_avg_category_3_B_mean,all_auth1_monthly_avg_category_3_C_mean,all_auth1_monthly_avg_month_lag_mean,all_auth1_monthly_avg_month_lag_std,all_auth1_monthly_avg_yyyy_week_nunique,all_auth1_monthly_avg_month_diff_mean,all_auth1_monthly_avg_month_diff_std,all_auth1_monthly_avg_purchase_amount_sum,all_auth1_monthly_avg_purchase_amount_max,all_auth1_monthly_avg_purchase_amount_min,all_auth1_monthly_avg_purchase_amount_mean,all_auth1_monthly_avg_installments_sum,all_auth1_monthly_avg_installments_max,all_auth1_monthly_avg_installments_min,all_auth1_monthly_avg_installments_mean,all_auth1_monthly_avg_installments_std,all_auth1_monthly_avg_merchant_id_nunique,all_auth1_monthly_avg_merchant_category_id_nunique,all_auth1_monthly_avg_card_id_size,all_auth1_monthly_avg_city_id_nunique,all_auth1_monthly_avg_state_id_nunique,all_auth1_monthly_avg_subsector_id_nunique,all_auth0_category_1_sum,all_auth0_category_1_mean,all_auth0_category_2_1.0_mean,all_auth0_category_2_2.0_mean,all_auth0_category_2_3.0_mean,all_auth0_category_2_4.0_mean,all_auth0_category_2_5.0_mean,all_auth0_category_3_A_mean,all_auth0_category_3_B_mean,all_auth0_category_3_C_mean,all_auth0_month_lag_mean,all_auth0_month_lag_std,all_auth0_yyyy_week_nunique,all_auth0_purchase_date_max,all_auth0_purchase_date_min,all_auth0_month_diff_mean,all_auth0_month_diff_std,all_auth0_purchase_amount_sum,all_auth0_purchase_amount_max,all_auth0_purchase_amount_min,all_auth0_purchase_amount_mean,all_auth0_installments_sum,all_auth0_installments_max,all_auth0_installments_min,all_auth0_installments_mean,all_auth0_installments_std,all_auth0_merchant_id_nunique,all_auth0_merchant_category_id_nunique,...,all_auth0_monthly_avg_purchase_amount_max,all_auth0_monthly_avg_purchase_amount_min,all_auth0_monthly_avg_purchase_amount_mean,all_auth0_monthly_avg_installments_sum,all_auth0_monthly_avg_installments_max,all_auth0_monthly_avg_installments_min,all_auth0_monthly_avg_installments_mean,all_auth0_monthly_avg_installments_std,all_auth0_monthly_avg_merchant_id_nunique,all_auth0_monthly_avg_merchant_category_id_nunique,all_auth0_monthly_avg_card_id_size,all_auth0_monthly_avg_city_id_nunique,all_auth0_monthly_avg_state_id_nunique,all_auth0_monthly_avg_subsector_id_nunique,all_new_category_1_sum,all_new_category_1_mean,all_new_category_2_1.0_mean,all_new_category_2_2.0_mean,all_new_category_2_3.0_mean,all_new_category_2_4.0_mean,all_new_category_2_5.0_mean,all_new_category_3_A_mean,all_new_category_3_B_mean,all_new_category_3_C_mean,all_new_month_lag_mean,all_ne

In [31]:
# Null Impute
for col in df_feat.columns:
    if col in ignore_list:continue
    if str(type(df_feat[col].dtypes)).count('time'):
        df_feat[col].fillna(df_feat[col].min(), inplace=True)

df_feat.fillna(-1, inplace=True)

### Save Feature

In [32]:
fname = '190_fe1'
ignore_features = ['first_active_month', 'card_id', target]

df_feat.columns = [col.replace('.', '_') for col in df_feat.columns]

for col in df_feat.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
#     if not(col.count('_diff')):continue
    feature = df_feat[col].astype('float32').values
    
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)

In [ ]:
# base = df_hist[[key, 'all_term']].drop_duplicates()
# utils.to_pkl_gzip(obj=base['all_term'].values,  path='../features/1_first_valid/190_fe1_all_term')
sys.exit()
import glob
path_list = glob.glob('../features/1_first_valid/*.gz')
for path in path_list:
    shape = utils.read_pkl_gzip(path).shape
    print(shape)